In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup

chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--no-sandbox")  # Bypass OS security model, required for headless mode
chrome_options.add_argument("--disable-gpu")  # Disable GPU hardware acceleration
chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems

driver = webdriver.Chrome(options=chrome_options)

In [12]:
driver.get('https://www.bbc.com/search?q=abortion')

In [13]:
#wait = WebDriverWait(driver, 10)
#load_more_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'div.button.load-more')))
#obstructing_element = WebDriverWait(driver, 10).until(
#    EC.invisibility_of_element((By.CSS_SELECTOR, 'div.alert-banner.is-breaking'))
#)

In [14]:
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="ddbddb249044d18fe4b52ce72e3c30f1")>

In [15]:
url = 'https://www.bbc.com/search?q=abortion'
link_set = set()
try:
    driver.get(url)
    # Wait for the elements inside the specific class to be present
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, 'sc-32f23d22-1 VSDHh')]//a[@data-testid='internal-link']"))
    )

    # Find all links within the specified class
    link_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'sc-32f23d22-1 VSDHh')]//a[@data-testid='internal-link']")
    for link_element in link_elements:
        link = link_element.get_attribute('href')
        print("Found link:", link)
        link_set.add(str(link))

finally:
    driver.quit()

Found link: https://www.bbc.com/news/health-68036171
Found link: https://www.bbc.com/news/world-latin-america-68014699
Found link: https://www.bbc.com/news/uk-politics-68033947
Found link: https://www.bbc.com/news/world-us-canada-68027344
Found link: https://www.bbc.com/news/uk-england-tees-67927099
Found link: https://www.bbc.com/news/uk-northern-ireland-67896139
Found link: https://www.bbc.com/news/world-australia-68064850
Found link: https://www.bbc.com/news/uk-northern-ireland-67749785
Found link: https://www.bbc.com/news/uk-england-dorset-67731454


In [16]:
link

'https://www.bbc.com/news/uk-england-dorset-67731454'

In [17]:
text_res = []
for link in link_set:
    if link != 'None':
        response = requests.get(link)
        print(link)
        soup = BeautifulSoup(response.content, 'html.parser')
        cur_text = []
        paragraphs = soup.find_all('p')
        for paragraph in paragraphs:
            cur_text.append(paragraph.get_text())
        text_res.append('\n'.join(cur_text))

https://www.bbc.com/news/uk-northern-ireland-67896139
https://www.bbc.com/news/health-68036171
https://www.bbc.com/news/uk-england-dorset-67731454
https://www.bbc.com/news/world-latin-america-68014699
https://www.bbc.com/news/uk-politics-68033947
https://www.bbc.com/news/uk-northern-ireland-67749785
https://www.bbc.com/news/world-us-canada-68027344
https://www.bbc.com/news/uk-england-tees-67927099
https://www.bbc.com/news/world-australia-68064850


In [18]:
with open('BBC_news.txt', 'w') as file:
    # Iterate over the list
    for item in text_res:
        # Write each item to the file followed by a newline
        file.write(item + '\n')